## 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor # 분석 알고리즘 DecisionTree 구현 라이브러리
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression # 변동성이 약한 데이터에서는 강한 모습을 보인다.
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split # 과거 데이터를 8:2나 7:3으로 자동으로 나눠주는 함수
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error # 분석 평가 대표지표 : MAE
from sklearn.metrics import mean_squared_error # 분석 평가 대표지표 : RMSE
# 머신러닝 라이브러니
# from sklearn.svm import SVC # 서포트 vector
from sklearn import svm # 서포트 vector
from sklearn.neighbors import KNeighborsClassifier # k neighbors
# 데이터 전처리 라이브러리
from sklearn.model_selection import train_test_split
# 클래시피케이션 metricx 
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score # 정확도 지표들
# 결과를 출력하기 위한 라이브러리
from sklearn.metrics import classification_report

## 데이터 불러오기 및 확인하기

In [2]:
htData = \
pd.read_csv("../../stddev/archive/heart.csv", encoding = "ms949")

In [3]:
htData.reset_index(inplace=True)

In [4]:
htData

,index,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [5]:
htData.count()

index       303
age         303
sex         303
cp          303
trtbps      303
chol        303
fbs         303
restecg     303
thalachh    303
exng        303
oldpeak     303
slp         303
caa         303
thall       303
output      303
dtype: int64

## 컬럼 간 상관관계 파악하기

In [6]:
corrDf = htData.corr()
corrDf

In [8]:
standardLimit = 0.4

In [9]:
# ['cp', 'thalachh', 'exng', 'oldpeak']
features = list(corrDf.loc[(abs(corrDf['output']) > standardLimit) & (corrDf['output'] != 1)].index)

for col in features:
    if col == 'index':
        features.remove(col)

In [10]:
lable = ['output']

## 데이터 분리

In [11]:
htData.shape

(303, 15)

In [12]:
seperation = 0.8
selectedIndex = int(list(htData.shape)[0] * seperation)
indexStd = htData['index'].loc[selectedIndex]
indexStd

242

In [13]:
features_data80,\
features_data20,\
label_data80,\
label_data20 = train_test_split(htData.loc[:,features],\
                                htData.loc[:,lable],test_size=0.2,random_state=5)

## 1.모델적용(의사결정 나무)

In [14]:
model = DecisionTreeClassifier(random_state=5)
model.fit(X=features_data80 ,y=label_data80)

DecisionTreeClassifier(random_state=5)

## 예측(의사결정 나무)

In [15]:
predictValue = model.predict(features_data20)
predictDF = pd.DataFrame(list(predictValue),columns=['PREDICT'])
validateDf = pd.concat([label_data20.reset_index(drop=True),predictDF],axis=1)
validateDf

,output,PREDICT
0,0,1
1,0,0
2,0,0
3,0,0
4,1,0
...,...,...
56,1,1
57,0,1
58,0,1
59,1,0


## 정확도 검증(의사결정 나무)

In [16]:
# MAE 평균절대오차
mae = mean_absolute_error(y_true=validateDf.output,
                   y_pred=validateDf.PREDICT)

# RMSE는 평균제곱근오차 MAE와 유사
# RMSE는 MAE에 비해 직관성은 떨어지지만, 로버스트한 성격(=극단적이지 않은)에서 강점을 보인다고 정리할 수 있다.
# "큰 오류값 차이에 대해서 크게 패널티를 주는" 이점이 있음
rmse = np.sqrt(mean_squared_error(y_true=validateDf.output,
                   y_pred=validateDf.PREDICT))
mae,rmse

(0.2786885245901639, 0.5279095799378564)

In [17]:
# 정확도 컬럼 추가
acc = pd.DataFrame(list(np.where( validateDf.output == validateDf.PREDICT, '성공', '실패')),columns=['acc'])
realValidateDf = pd.concat([validateDf.reset_index(drop=True),acc],axis=1)
realValidateDf

,output,PREDICT,acc
0,0,1,실패
1,0,0,성공
2,0,0,성공
3,0,0,성공
4,1,0,실패
...,...,...,...
56,1,1,성공
57,0,1,실패
58,0,1,실패
59,1,0,실패


In [18]:
realAcc = realValidateDf.loc[realValidateDf.acc=='성공'].shape[0]/realValidateDf.shape[0]
realAcc

0.7213114754098361

## 2.모델적용(랜덤 포레스트)

In [19]:
model2 = RandomForestClassifier(random_state=5)
model2.fit(X=features_data80, y=label_data80)

<ipython-input-19-ece03eb253d2>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model2.fit(X=features_data80, y=label_data80)


RandomForestClassifier(random_state=5)

## 예측(랜덤 포레스트)

In [20]:
predictValue2 = model2.predict(features_data20)
predictDF2 = pd.DataFrame(list(predictValue2),columns=['PREDICT2'])
validateDf2 = pd.concat([label_data20.reset_index(drop=True),predictDF2],axis=1)
validateDf2

,output,PREDICT2
0,0,1
1,0,0
2,0,0
3,0,0
4,1,1
...,...,...
56,1,1
57,0,1
58,0,0
59,1,0


## 정확도 검증(랜덤 포레스트)

In [21]:
# MAE 평균절대오차
mae2 = mean_absolute_error(y_true=validateDf2.output,
                   y_pred=validateDf2.PREDICT2)

# RMSE는 평균제곱근오차 MAE와 유사
# RMSE는 MAE에 비해 직관성은 떨어지지만, 로버스트한 성격(=극단적이지 않은)에서 강점을 보인다고 정리할 수 있다.
# "큰 오류값 차이에 대해서 크게 패널티를 주는" 이점이 있음
rmse2 = np.sqrt(mean_squared_error(y_true=validateDf2.output,
                   y_pred=validateDf2.PREDICT2))
mae2,rmse2

(0.16393442622950818, 0.40488816508945796)

In [22]:
# 정확도 컬럼 추가
acc2 = pd.DataFrame(list(np.where( validateDf2.output == validateDf2.PREDICT2, '성공', '실패')),columns=['acc2'])
realValidateDf2 = pd.concat([validateDf2.reset_index(drop=True),acc2],axis=1)
realValidateDf2

,output,PREDICT2,acc2
0,0,1,실패
1,0,0,성공
2,0,0,성공
3,0,0,성공
4,1,1,성공
...,...,...,...
56,1,1,성공
57,0,1,실패
58,0,0,성공
59,1,0,실패


In [23]:
realAcc2 = realValidateDf2.loc[realValidateDf2.acc2=='성공'].shape[0]/realValidateDf2.shape[0]
realAcc2

0.8360655737704918

## 3.모델적용(선형회귀)

In [24]:
model3 = LinearRegression()
model3.fit(X=features_data80 ,y=label_data80)

LinearRegression()

In [25]:
model3.coef_

array([[ 0.11287352,  0.00444584, -0.20976594, -0.10447137]])

In [26]:
model3.intercept_

array([-0.06051548])

## 예측(선형회귀)

In [27]:
predictValue3 = model3.predict(features_data20)
predictDF3 = pd.DataFrame(list(predictValue3),columns=['PREDICT3'])
validateDf3 = pd.concat([label_data20.reset_index(drop=True),predictDF3],axis=1)
validateDf3

,output,PREDICT3
0,0,0.825934
1,0,-0.004413
2,0,-0.088873
3,0,0.274339
4,1,0.713438
...,...,...
56,1,0.730795
57,0,0.480490
58,0,0.442673
59,1,0.832107


## 정확도 검증(선형회귀)

In [28]:
# MAE 평균절대오차
mae3 = mean_absolute_error(y_true=validateDf3.output,
                   y_pred=validateDf3.PREDICT3)

# RMSE는 평균제곱근오차 MAE와 유사
# RMSE는 MAE에 비해 직관성은 떨어지지만, 로버스트한 성격(=극단적이지 않은)에서 강점을 보인다고 정리할 수 있다.
# "큰 오류값 차이에 대해서 크게 패널티를 주는" 이점이 있음
rmse3 = np.sqrt(mean_squared_error(y_true=validateDf3.output,
                   y_pred=validateDf3.PREDICT3))
mae3,rmse3

(0.2762424818403152, 0.34349851605744847)

## 4.모델적용(서포트백터)

In [29]:
model4= svm.SVC(random_state=5)
model4.fit( X=features_data80,y=label_data80)

C:\Users\SMART-19\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVC(random_state=5)

## 예측(서포트백터)

In [30]:
predictValue4 = model4.predict(features_data20)
predictDF4 = pd.DataFrame(list(predictValue4),columns=['PREDICT4'])
validateDf4 = pd.concat([label_data20.reset_index(drop=True),predictDF4],axis=1)
validateDf4

,output,PREDICT4
0,0,1
1,0,0
2,0,0
3,0,1
4,1,1
...,...,...
56,1,1
57,0,1
58,0,1
59,1,1


## 정확도 검증(서포트백터)

In [31]:
# MAE 평균절대오차
mae4 = mean_absolute_error(y_true=validateDf4.output,
                   y_pred=validateDf4.PREDICT4)

# RMSE는 평균제곱근오차 MAE와 유사
# RMSE는 MAE에 비해 직관성은 떨어지지만, 로버스트한 성격(=극단적이지 않은)에서 강점을 보인다고 정리할 수 있다.
# "큰 오류값 차이에 대해서 크게 패널티를 주는" 이점이 있음
rmse4 = np.sqrt(mean_squared_error(y_true=validateDf4.output,
                   y_pred=validateDf4.PREDICT4))
mae4,rmse4

(0.29508196721311475, 0.5432144762551112)

In [32]:
# 정확도 컬럼 추가
acc4 = pd.DataFrame(list(np.where( validateDf4.output == validateDf4.PREDICT4, '성공', '실패')),columns=['acc4'])
realValidateDf4 = pd.concat([validateDf4.reset_index(drop=True),acc4],axis=1)
realValidateDf4

,output,PREDICT4,acc4
0,0,1,실패
1,0,0,성공
2,0,0,성공
3,0,1,실패
4,1,1,성공
...,...,...,...
56,1,1,성공
57,0,1,실패
58,0,1,실패
59,1,1,성공


In [33]:
realAcc4 = realValidateDf4.loc[realValidateDf4.acc4=='성공'].shape[0]/realValidateDf4.shape[0]
realAcc4

0.7049180327868853